# Data generation
Generates a dataset with different
- slopes
- hole sizes
- hole locations

In [29]:
import numpy as np

# structure
# [datawidth] columns of height data -100 for null
# hole present (0=no or 1=yes)
# hole size (0 for not present, 1 for one pixel wide hole)


# build examples
dataWidth = 6
minSlope = -3
maxSlope = 3
nullValue = -100

# build varying slopes
def generateSlopes():
    dataset = None
    i=0
    slope = np.empty(maxSlope-minSlope+1)
    for currentSlope in range (minSlope, maxSlope+1):
        slope[i] = currentSlope
        lastValue = 0
        if (currentSlope == 0):
            newdata = np.zeros(dataWidth)
        else:
            newdata = np.arange(0, dataWidth*currentSlope, currentSlope)
        if (dataset is None):
            dataset = newdata
        else:
            dataset = np.vstack((dataset, newdata))
        i+=1
    return dataset    

def knockOutHoles(dataset):
    ds1 = dataset.copy()
    ds2 = dataset.copy()
    ds3 = dataset.copy()
    for row in ds1:
        row[2] = nullValue
    for row in ds2:
        row[3] = nullValue
    for row in ds3:
        row[2:4] = nullValue
    return np.concatenate((ds1, ds2, ds3))

def trimLeftEdge(dataset):
    for row in dataset:
        row[0] = nullValue
    return dataset

def trimRightEdge(dataset):
    for row in dataset:
        row[-1] = nullValue
    return dataset

def holeData(row):
    lastValue = nullValue
    steppedDown = False
    holeSize = 0
    for value in row:
        # check if we stepped down to null
        if (steppedDown):
            holeSize = holeSize+1

            # check for step up after having stepped down
            if (lastValue == nullValue and value != nullValue):
                return True, holeSize
                steppedDown = False
        else:
            # check for step down to null
            if (lastValue != nullValue and value == nullValue):
                steppedDown = True

        lastValue = value

    # no hole found
    return False, 0

def buildLabels(unlabled):
    nRows = np.shape(unlabled)[0]
    holePresent = np.zeros((nRows,1))
    holeSize = np.zeros((nRows,1))
    i = 0
    for row in unlabled:
        h = holeData(row)
        holePresent[i] = h[0]
        holeSize[i] = h[1]           
        i = i+1
    lables = np.hstack((holePresent,holeSize))
    return lables


ds = generateSlopes()
ds = np.concatenate((ds, knockOutHoles(ds.copy())))
ds = np.concatenate((ds, trimLeftEdge(ds.copy())))
ds = np.concatenate((ds, trimRightEdge(ds.copy())))

np.shape(ds)

lables = buildLabels(ds)
lds = np.hstack((ds, lables))
# format output
headerStr = ""
for i in range(dataWidth):
    headerStr = headerStr + "v" + str(i) + ", "
headerStr = headerStr + " holePresent, holeWidth"
print("HeaderStr: "+headerStr)
np.savetxt("holes6.csv", lds, delimiter=",", header=headerStr)
print(lds)

HeaderStr: v0, v1, v2, v3, v4, v5,  holePresent, holeWidth
[[   0.   -3.   -6.   -9.  -12.  -15.    0.    0.]
 [   0.   -2.   -4.   -6.   -8.  -10.    0.    0.]
 [   0.   -1.   -2.   -3.   -4.   -5.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    1.    2.    3.    4.    5.    0.    0.]
 [   0.    2.    4.    6.    8.   10.    0.    0.]
 [   0.    3.    6.    9.   12.   15.    0.    0.]
 [   0.   -3. -100.   -9.  -12.  -15.    1.    1.]
 [   0.   -2. -100.   -6.   -8.  -10.    1.    1.]
 [   0.   -1. -100.   -3.   -4.   -5.    1.    1.]
 [   0.    0. -100.    0.    0.    0.    1.    1.]
 [   0.    1. -100.    3.    4.    5.    1.    1.]
 [   0.    2. -100.    6.    8.   10.    1.    1.]
 [   0.    3. -100.    9.   12.   15.    1.    1.]
 [   0.   -3.   -6. -100.  -12.  -15.    1.    1.]
 [   0.   -2.   -4. -100.   -8.  -10.    1.    1.]
 [   0.   -1.   -2. -100.   -4.   -5.    1.    1.]
 [   0.    0.    0. -100.    0.    0.    1.    1.]
 [   0.    1.    2. -10

# Learning
Fit a model to the data

In [2]:

# first neural network with keras make predictions
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
# load the dataset
dataset = loadtxt('holes6.csv', delimiter=',')
# split into input (X) and output (y) variables
X = dataset[:,0:6]
y = dataset[:,7]
# define the keras model
model = Sequential()
model.add(Dense(36, input_dim=6, activation='relu'))
model.add(Dense(30, activation='sigmoid'))
model.add(Dense(20, activation='relu'))
model.add(Dense(1, activation='relu'))
# compile the keras model
print("Compiling model...")
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])



Compiling model...


In [4]:
# fit the keras model on the dataset
print("Fitting model...")
history = model.fit(X, y, epochs=20, batch_size=20, verbose=0)


Fitting model...


In [6]:
# list all data in history
print(history.history.keys())


dict_keys(['loss', 'accuracy'])


In [7]:
import matplotlib.pyplot as plt
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
#plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

NameError: name 'plt' is not defined

In [ ]:

# make class predictions with the model
print("Making predictions...")
predictions = model.predict(X)


In [ ]:

error = np.subtract(y,predictions)
error = np.hstack((error, range(len(y))))

# summarize the first 5 cases
for i in range(100):
	print('%s => %.1f (expected %.1f)' % (X[i].tolist(), predictions[i], y[i]))